In [1]:
import pandas as pd

df = pd.read_csv("ul_dec30_recoding_all.csv")

# Filter rows where 'summary' contains "ADHD" 
adhd_apps = df[df['summary'].str.contains('ADHD', case=False, na=False)]

# Display the number of ADHD-related apps
print(f"Number of apps mentioning ADHD in summary: {len(adhd_apps)}")



Number of apps mentioning ADHD in summary: 0


,title,summary


In [4]:
google_play_df = df[df['store'] == 'Google Play']

google_play_df.to_csv('google_paper_file.csv', index=False)

print("Filtering complete. Saved to 'google_paper_file.csv'.")



apple_store_df = df[df['store'] == 'Apple App Store']

apple_store_df.to_csv('apple_paper_file.csv', index=False)

print("Filtering complete. Saved to 'apple_paper_file.csv'.")


chrome_df = df[df['store'] == 'Chrome Web Store']

chrome_df.to_csv('chrome_paper_file.csv', index=False)

print("Filtering complete. Saved to 'chrome_paper_file.csv'.")


Filtering complete. Saved to 'google_paper_file.csv'.
Filtering complete. Saved to 'apple_paper_file.csv'.
Filtering complete. Saved to 'chrome_paper_file.csv'.


In [11]:
import re
import csv
import time
from google_play_scraper import reviews, Sort

input_file = "google_paper_file.csv"
output_file = "google_paper_adhd.csv"
adhd_keyword = "adhd"

def extract_package_name(url):
    match = re.search(r'id=([a-zA-Z0-9._]+)', url)
    return match.group(1) if match else None

with open(input_file, 'r', newline='', encoding='utf-8') as infile, \
     open(output_file, 'w', newline='', encoding='utf-8') as outfile:

    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames + ['reviews']  # Add a new column for ADHD reviews
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        url = row.get('url', '')
        package_name = extract_package_name(url)
        if not package_name:
            print(f"⚠️ Skipping row with invalid URL: {url}")
            continue

        print(f"📝 Fetching reviews for: {row.get('title', '[No Title]')} ({package_name})")
        matched_reviews = []

        try:
            review_data, _ = reviews(
                package_name,
                lang='en',
                country='us',
                sort=Sort.NEWEST,
            )

            for r in review_data:
                if adhd_keyword in r['content'].lower():
                    matched_reviews.append(r['content'])

        except Exception as e:
            print(f"⚠️ Error fetching reviews for {package_name}: {e}")
            continue

        if matched_reviews:
            row['reviews'] = "\n---\n".join(matched_reviews)
            writer.writerow(row)
            print(f"✔️ Saved {len(matched_reviews)} ADHD review(s)")

        time.sleep(1)

print(f"\n✅ ADHD-related reviews saved to: {output_file}")


📝 Fetching reviews for: ( OFFTIME ) - Distraction Free (co.offtime.kit)
📝 Fetching reviews for: Absolock | Phone Locking app (olsencheung.lockyourphoneproject)
📝 Fetching reviews for: Addicted - phone usage tracker (com.tobiasschuerg.smartphonestats)
📝 Fetching reviews for: Anti Smartphone Addiction (com.antiaddication.innomotion)
📝 Fetching reviews for: AntiSocial: phone addiction (com.goozix.antisocial_personal)
📝 Fetching reviews for: App Blocker (Stay Focused) (com.appblockernew)
📝 Fetching reviews for: App Gatekeeper -Curb Addiction (com.timelesssky.appgatekeeper)
📝 Fetching reviews for: App Off Timer (com.aozora_create.appofftimer)
📝 Fetching reviews for: App Usage - Manage/Track Usage (com.a0soft.gphone.uninstaller)
📝 Fetching reviews for: AppBlock - Stay Focused (cz.mobilesoft.appblock)
📝 Fetching reviews for: AppDetox - App Blocker for Digital Detox (de.dfki.appdetox)
📝 Fetching reviews for: Appdicted - Apps tracker (gfg.app.appdicted)
📝 Fetching reviews for: Appology (bis.mcc

Extracted package name: co.offtime.kit
